<a href="https://colab.research.google.com/github/vishwakarma9470/Machine_learning_Project/blob/main/job_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle


In [ ]:
df = pd.read_csv("naukri_com-job_sample.csv", low_memory=False)
df.head()

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id,Unnamed: 14,Unnamed: 15,Unnamed: 16,skills.1
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,2.105160e+11,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8,NaN,NaN,NaN,Tech
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,2.105160e+11,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0,NaN,NaN,NaN,Non -Tech
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,1.010170e+11,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba,NaN,NaN,NaN,Tech
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,8.101690e+10,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae,NaN,NaN,NaN,Non - Tech
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,1.209160e+11,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4,NaN,NaN,NaN,Tech


In [ ]:
df.fillna('', inplace=True)

<ipython-input-15-fe4ba46ae7c4>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [ ]:
def recommend_jobs(user_education, user_skills=None, top_n=5):
    df['combined_features'] = (
        df['education'] + ' ' +
        df['skills'] + ' ' +
        df['jobdescription']
    )

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['combined_features'])

    user_input = user_education
    if user_skills:
        user_input += ' ' + user_skills
    user_tfidf = tfidf.transform([user_input])

    cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)

    # Get top job recommendations
    top_indices = cosine_sim.argsort()[0][::-1][:top_n]
    recommendations = df.iloc[top_indices][[
        'company', 'jobtitle', 'joblocation_address', 'education', 'skills', 'jobdescription'
    ]]
    return recommendations

In [ ]:
user_education = "UG: B.Tech/B.E. - Computer Science PG:MBA - Finance"
user_skills = "Python, Machine Learning"
recommendations = recommend_jobs(user_education, user_skills, top_n=5)
recommendations_df = recommendations.reset_index(drop=True)
recommendations_df

,company,jobtitle,joblocation_address,education,skills,jobdescription
0,QUORA,Software Engineer - Machine Learning,Mumbai,UG: B.Tech/B.E. PG:M.Tech Doctorate:Any Doctor...,IT Software - Telecom Software,Job Description Send me Jobs like this espon...
1,Startup - CloudInfra,Software Engineer,Bengaluru,"UG: Any Graduate - Any Specialization, B.Tech/...",IT Software - Application Programming,Job Description Send me Jobs like this Come ...
2,QUORA,Engineering Manager - Machine Learning,,UG: Any Graduate PG:Post Graduation Not Required,Production,Job Description Send me Jobs like this Respo...
3,LiveOnGo,Senior Software Engineer- Startup,Bengaluru/Bangalore,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,IT Software - Application Programming,Job Description Send me Jobs like this Job D...
4,Career Maker,Research Scientist/engineer - Computer Vision/...,Hyderabad,"UG: Any Graduate - Any Specialization, B.Tech/...",IT Software - Application Programming,Job Description Send me Jobs like this Resea...


In [ ]:
user_education = "B.Tech/B.E"
user_skills = "Python, Machine Learning"
recommendations = recommend_jobs(user_education, user_skills, top_n=5)
recommendations_df = recommendations.reset_index(drop=True)
recommendations_df

,company,jobtitle,joblocation_address,education,skills,jobdescription
0,QUORA,Software Engineer - Machine Learning,Mumbai,UG: B.Tech/B.E. PG:M.Tech Doctorate:Any Doctor...,IT Software - Telecom Software,Job Description Send me Jobs like this espon...
1,Startup - CloudInfra,Software Engineer,Bengaluru,"UG: Any Graduate - Any Specialization, B.Tech/...",IT Software - Application Programming,Job Description Send me Jobs like this Come ...
2,Career Maker,Sr Scientist/ NLP/ Machine Learning/ Data Mini...,Bengaluru/Bangalore,UG: Any Graduate - Any Specialization PG:Any P...,Analytics & Business Intelligence,Job Description Send me Jobs like this Major...
3,Careernet Technologies Pvt Ltd hiring for Prod...,Data Scientist,Bengaluru,PG:M.Tech - Any Specialization Doctorate:Ph.D ...,IT Software - eCommerce,Job Description Send me Jobs like this Brief...
4,QUORA,Engineering Manager - Machine Learning,,UG: Any Graduate PG:Post Graduation Not Required,Production,Job Description Send me Jobs like this Respo...
